In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#import tensorflow.contrib.keras as keras
#! pip install tensorflow --upgrade --user
#! pip install keras --user
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from keras import model
#from keras import layer

In [22]:
data = pd.read_csv('datatest2.txt')
data.head(5)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9752 entries, 1 to 9752
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           9752 non-null   object 
 1   Temperature    9752 non-null   float64
 2   Humidity       9752 non-null   float64
 3   Light          9752 non-null   float64
 4   CO2            9752 non-null   float64
 5   HumidityRatio  9752 non-null   float64
 6   Occupancy      9752 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 609.5+ KB


In [23]:
def toOrdinalDate(column):
    arr=[]
    for date in column:
        time=pd.Timestamp(date)
        time=time.toordinal()
        arr.append(time)
    return pd.DataFrame(np.array(arr), index=data.index)
data['date'] = toOrdinalDate(data['date'].to_numpy())

In [24]:
def rescaling(dfColumn):
    feature = dfColumn.to_numpy().reshape(-1, 1)
    minmax_scale = StandardScaler()
    scaled_feature = minmax_scale.fit_transform(feature)
    return pd.Series(scaled_feature.reshape(-1), index=data.index)

for column in data:
    if column != 'Occupancy':
        data[column] = rescaling(data[column])


In [25]:
data.head(5)

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,-1.755628,0.742898,0.314074,1.509363,0.930527,0.814063,1
2,-1.755628,0.772291,0.280342,1.509363,0.830667,0.790653,1
3,-1.755628,0.750246,0.311334,1.493354,0.843289,0.815115,1
4,-1.755628,0.750246,0.311334,1.517368,0.862644,0.815115,1
5,-1.755628,0.772291,0.314074,1.509363,0.849741,0.831554,1


In [30]:
from sklearn.model_selection import train_test_split
X = data.iloc[:,0:5 ]
Y = data.iloc[:,-1]
#print(X)
#print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

          date  Temperature  Humidity     Light       CO2
1    -1.755628     0.742898  0.314074  1.509363  0.930527
2    -1.755628     0.772291  0.280342  1.509363  0.830667
3    -1.755628     0.750246  0.311334  1.493354  0.843289
4    -1.755628     0.750246  0.311334  1.517368  0.862644
5    -1.755628     0.772291  0.314074  1.509363  0.849741
...        ...          ...       ...       ...       ...
9748  1.752750    -0.182991 -0.550116  1.472942  2.531382
9749  1.752750    -0.134003 -0.543158  1.442924  2.562518
9750  1.752750    -0.109508 -0.543158  1.442924  2.586081
9751  1.752750    -0.109508 -0.472952  1.420111  2.958033
9752  1.752750    -0.001733 -0.453345  1.373283  3.738966

[9752 rows x 5 columns]
1       1
2       1
3       1
4       1
5       1
       ..
9748    1
9749    1
9750    1
9751    1
9752    1
Name: Occupancy, Length: 9752, dtype: int64


In [35]:
network = Sequential()
network.add(Dense(units=16, activation="relu"))
network.add(Dense(units=16,activation="relu"))
network.add(Dense(units=1, activation="sigmoid"))
network.compile(optimizer='sgd',  
                   loss='binary_crossentropy', 
                   metrics=['accuracy'])
history = network.fit(X_train, Y_train, 
                      batch_size=100, epochs=30, 
                      validation_data=(X_test, Y_test))

Epoch 1/30
79/79 [==============================] - 2s 12ms/step - loss: 0.6570 - accuracy: 0.7192 - val_loss: 0.5741 - val_accuracy: 0.7868
Epoch 2/30
79/79 [==============================] - 1s 9ms/step - loss: 0.5451 - accuracy: 0.7791 - val_loss: 0.4736 - val_accuracy: 0.7914
Epoch 3/30
79/79 [==============================] - 1s 11ms/step - loss: 0.4451 - accuracy: 0.8131 - val_loss: 0.3819 - val_accuracy: 0.8662
Epoch 4/30
79/79 [==============================] - 1s 9ms/step - loss: 0.3501 - accuracy: 0.8903 - val_loss: 0.3038 - val_accuracy: 0.9195
Epoch 5/30
79/79 [==============================] - 1s 8ms/step - loss: 0.2733 - accuracy: 0.9491 - val_loss: 0.2399 - val_accuracy: 0.9544
Epoch 6/30
79/79 [==============================] - 1s 8ms/step - loss: 0.2147 - accuracy: 0.9668 - val_loss: 0.1926 - val_accuracy: 0.9662
Epoch 7/30
79/79 [==============================] - 1s 7ms/step - loss: 0.1679 - accuracy: 0.9773 - val_loss: 0.1600 - val_accuracy: 0.9672
Epoch 8/30
79/79 [